# Import packages and set parameters

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

## Gap detection in DOS

In [ ]:
@jit(nopython=True)
def gap_detector(fermis, IDS, min_IDS_width = 0.001, min_energy_width = 0.1):
    gaps = np.zeros((0,2),dtype=np.float64)
    
    i = 0
    while i <= IDS.size:

        j = 1
        while abs(IDS[i+j]-IDS[i]) < min_IDS_width:
            if j+i == IDS.size-1:
                break
            else:
                j +=1
        else:
            if abs(fermis[i+j]-fermis[i]) > min_energy_width:
                gaps = np.append(gaps,np.array([[fermis[i],fermis[i+j]]]),axis=0)
                i += j
                continue
            else:
                i +=1
                continue
        
        return gaps

## Chern coefficients

In [ ]:
def n_(theta,Ch_,Ch_u1t1):
    return Ch_ - theta * Ch_u1t1

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if data.at[0,'texture'] != "limacon":
        continue
    
    if "n_fermi" in data:
        df = pd.concat([df,data])
    if "n_moments" in data:
        df = pd.concat([df,data])
    if os.path.isfile(file+"/chern_numbers_tau1u1.npy"):
        Chern_numbers = np.load(file+"/chern_numbers_tau1u1.npy",allow_pickle=True)[()]

        for label in Chern_numbers:
            if label=="":
                data["IDS"]=Chern_numbers[label].real
            else:
                data[label]=Chern_numbers[label].real
        df = pd.concat([df,data])

display(df.set_index('key').sort_index(ascending=False))

## Select data

In [ ]:
key_spec, key_fermi_l, key_fermi_r = '1746537187', '1746286805', '1746308778' # theta_l=0.110 theta_r=0.257

gap_keys = ['1746179416','1746179418','1746179420','1746179422','1746179424','1746179426']

## Import file and extract data

In [ ]:
# parameter
size_l,  size_r  = df.set_index('key').at[key_fermi_l,'system_sizes'], df.set_index('key').at[key_fermi_r,'system_sizes']
theta_l, theta_r = df.set_index('key').at[key_fermi_l,'q'],            df.set_index('key').at[key_fermi_r,'q']

sys_size, n_q    = df.set_index('key').at[key_spec,'system_sizes'], df.set_index('key').at[key_spec,'n_q']
t,       m       = df.set_index('key').at[key_spec,'t'],               df.set_index('key').at[key_spec,'m']

n_energies, n_moments, n_random_states = df.set_index('key').at[key_spec,'n_energies'], df.set_index('key').at[key_spec,'n_moments'], df.set_index('key').at[key_spec,'n_random_states']
scale, epsilon = df.set_index('key').at[key_spec,'scale'], df.set_index('key').at[key_spec,'epsilon']

n_fermi_l, min_fermi_l, max_fermi_l = df.set_index('key').at[key_fermi_l,'n_fermi'],  df.set_index('key').at[key_fermi_l,'min_fermi'], df.set_index('key').at[key_fermi_l,'max_fermi']
n_fermi_r, min_fermi_r, max_fermi_r = df.set_index('key').at[key_fermi_r,'n_fermi'],  df.set_index('key').at[key_fermi_r,'min_fermi'], df.set_index('key').at[key_fermi_r,'max_fermi']

# Data
outdir_fermi_l = './data/' + key_fermi_l
outdir_fermi_r = './data/' + key_fermi_r
outdir_spec    = './data/' + key_spec

# DOS
fermis_l = np.linspace(min_fermi_l,max_fermi_l,n_fermi_l)
fermis_r = np.linspace(min_fermi_r,max_fermi_r,n_fermi_r)

spectrum_l = np.load(outdir_fermi_l+"/spectrum.npy")
spectrum_r = np.load(outdir_fermi_r+"/spectrum.npy")

# Chern numbers
ch_0_l = np.array([])
for f in fermis_l:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum_l])
    ch_0_l = np.append(ch_0_l,[sum(occ)/size_l])
    
ch_u1t1_l  = np.load(outdir_fermi_l+"/ch_tau1u1.npy")

n_0_l    = n_(theta_l,ch_0_l,ch_u1t1_l)
n_u1t1_l = ch_u1t1_l

ch_0_r = np.array([])
for f in fermis_r:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum_r])
    ch_0_r = np.append(ch_0_r,[sum(occ)/size_r])
    
ch_u1t1_r  = np.load(outdir_fermi_r+"/ch_tau1u1.npy")

n_0_r    = n_(theta_r,ch_0_r,ch_u1t1_r)
n_u1t1_r = ch_u1t1_r
    
# data
qs = np.array([ (1 + i*(sys_size//n_q) ) /sys_size for i in range(0,math.floor(n_q/2))])

emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

n_qs = len(qs)
dos = []
for i in range(n_qs):
    dos.append( np.load('./data/'+key_spec+'/dos_'+str(i).zfill(4)+'.npy') )
dos = np.array(dos)

Emin = np.amin(emesh)
Emax = np.amax(emesh)

phimin = np.amin(qs)
phimax = np.amax(qs)

# gap markers
fermis = np.zeros(len(gap_keys))
thetas = np.zeros(len(gap_keys))
for i,k in enumerate(gap_keys):
    fermis[i] = df.set_index('key').at[k,'fermi']
    thetas[i] = df.set_index('key').at[k,'q']

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'cmu serif'

tfs    = 24 #title font size
lfs    = 24 #label font size
fs     = 22 #font size
cbarfs = 18 #colorbar font size
afs    = 24 #annotation font size
gfs    = 18 #gap font size

# scatter size
s = 2

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

# DoS + Chern

In [ ]:
gaps_l = gap_detector(fermis_l,ch_0_l,0.001,0.05)
gaps_r = gap_detector(fermis_r,ch_0_r,0.001,0.05)
print(gaps_l.shape,gaps_r.shape)
# print(gaps_l)
# print([sum(g)/2 for g in gaps_l])

In [ ]:
x_ticks1 = np.arange(0, 3, 1)
x_ticks2 = np.arange(-6, 8, 2)
x_ticks3 = np.arange(0.1, 0.5, 0.1)  
x_ticks4 = np.arange(0, 3, 1)
x_ticks5 = np.arange(-2, 3, 1)
y_ticks  = np.arange(-3, 4, 1)
    
fig = plt.figure()
gs  = fig.add_gridspec(1,24, hspace=0,wspace=0)
ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
ax2 = plt.subplot(gs.new_subplotspec((0, 3), colspan=6),sharey=ax1)
ax3 = plt.subplot(gs.new_subplotspec((0, 9), colspan=8),sharey=ax1)
ax4 = plt.subplot(gs.new_subplotspec((0, 17), colspan=3),sharey=ax1)
ax5 = plt.subplot(gs.new_subplotspec((0, 20), colspan=4),sharey=ax1)

ax1.scatter(n_0_l.real,fermis_l,s)
ax1.set_xticks(x_ticks1)
ax1.xaxis.set_minor_locator(MultipleLocator(0.5))
ax1.grid(True,which='minor',color='lightgray',axis='x')
ax1.set_xlim([-0.5,2.5])
ax1.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',labelpad=-10.0,fontsize=lfs)
ax1.set_title(r'$n_\emptyset$',fontsize=tfs,y=1.005)

ax2.scatter(n_u1t1_l.real,fermis_l,s)
ax2.set_xticks(x_ticks2)
ax2.xaxis.set_minor_locator(MultipleLocator(1))
ax2.grid(True,which='minor',color='lightgray',axis='x')
ax2.set_xlim([-7,7])
ax2.set_title(r'$n_{u_1 t_1}$',fontsize=tfs,y=1.005)

ax3.imshow(dos.T, aspect='auto',norm=norm, extent=(phimin,phimax, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax3.set_xticks(x_ticks3)
ax3.set_xlim([phimin,phimax])
ax3.xaxis.set_minor_locator(MultipleLocator(0.05))
ax3.set_title(r"DOS",fontsize=tfs,y=1.005)
ax3.set_xlabel(r"$\vartheta$",fontsize=lfs)

ax3.axvline(x = theta_l, color = 'red', label = r'$\theta_l$')
ax3.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta_l*size_l,size_l),(theta_l+0.01,3.55),color='red',fontsize=afs)
ax3.axvline(x = theta_r, color = 'green', label = r'$\theta_r$')
ax3.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta_r*size_r,size_r),(theta_r-0.11,-3.75),color='green',fontsize=afs)

ax3.annotate(r'$U_{-1}$',(0.01,3.15),color='black',fontsize=gfs)
ax3.annotate(r'$U_{-2}$',(0.01,2.65),color='black',fontsize=gfs)
ax3.annotate(r'$U_{-3}$',(0.01,2.2),color='black',fontsize=gfs)
ax3.annotate(r'$U_{-4}$',(0.01,1.75),color='black',fontsize=gfs)
ax3.annotate(r'$U_{-5}$',(0.01,1.38),color='black',fontsize=gfs)
ax3.annotate(r'$U_{-6}$',(0.01,1.1),color='black',fontsize=gfs)

ax3.annotate(r'$C_1$',(0.01,0.45),color='black',fontsize=gfs)
ax3.annotate(r'$C_0$',(0.01,-0.1),color='black',fontsize=gfs)
ax3.annotate(r'$C_{-1}$',(0.01,-0.6),color='black',fontsize=gfs)

ax3.annotate(r'$L_1$',(0.01,-3.3),color='black',fontsize=gfs)
ax3.annotate(r'$L_2$',(0.01,-2.85),color='black',fontsize=gfs)
ax3.annotate(r'$L_3$',(0.01,-2.4),color='black',fontsize=gfs)
ax3.annotate(r'$L_4$',(0.01,-1.85),color='black',fontsize=gfs)
ax3.annotate(r'$L_5$',(0.01,-1.48),color='black',fontsize=gfs)
ax3.annotate(r'$L_6$',(0.01,-1.12),color='black',fontsize=gfs)

ax3.annotate(r'$U_{-1}$',(0.45,2.55),color='black',fontsize=gfs)
ax3.annotate(r'$U_{-2}^\prime$',(0.45,1.8),color='black',fontsize=gfs)

ax3.annotate(r'$C_2^\prime$',(0.45,2.10),color='black',fontsize=gfs)
ax3.annotate(r'$C_1^\prime$',(0.45,0.95),color='black',fontsize=gfs)
ax3.annotate(r'$C_0^\prime$',(0.45,-0.1),color='black',fontsize=gfs)
ax3.annotate(r'$C_{-1}^\prime$',(0.45,-1.1),color='black',fontsize=gfs)
ax3.annotate(r'$C_{-2}^\prime$',(0.45,-2.25),color='black',fontsize=gfs)

ax3.annotate(r'$L_1$',(0.45,-2.65),color='black',fontsize=gfs)
ax3.annotate(r'$L_2^\prime$',(0.45,-1.95),color='black',fontsize=gfs)

ax4.scatter(n_0_r.real,fermis_r,s)
ax4.set_xticks(x_ticks4)
ax4.xaxis.set_minor_locator(MultipleLocator(0.5))
ax4.grid(True,which='minor',color='lightgray',axis='x')
ax4.set_xlim([-0.5,2.5])
ax4.set_title(r'$n_\emptyset$',fontsize=tfs,y=1.005)

ax5.scatter(ch_u1t1_r.real,fermis_r,s)
ax5.set_xticks(x_ticks5)
ax5.xaxis.set_minor_locator(MultipleLocator(1))
ax5.grid(True,which='minor',color='lightgray',axis='x')
ax5.set_xlim([-3,3])
ax5.set_title(r'$n_{u_1 t_1}$',fontsize=tfs,y=1.005)

for ax in fig.get_axes():
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_yticks(y_ticks)
    ax.set_ylim([-4,4])
    
for ax in fig.get_axes()[0:2]:
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps_l:
        ax.fill_between(np.arange(-15,15,0.01),g[0],g[1], facecolor='gray', alpha=0.5)
        
for g in gaps_l:
    ax3.fill_between(np.arange(0.0,theta_l,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
for g in gaps_r:
    ax3.fill_between(np.arange(theta_r,0.51,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
    
for ax in fig.get_axes()[3:5]:
    ax.yaxis.tick_right()
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps_r:
        ax.fill_between(np.arange(-15,15,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
        
for ax in fig.get_axes()[1:]:
    ax.yaxis.tick_right()
    
# attaching gap labels
gaps = np.array([(thetas[i],fermis[i]) for i in range(len(gap_keys))]) # s=0,s=pi , m=0.0

gaps_col = cm.plasma(np.linspace(0,1,len(gaps)))
ax3.scatter(gaps[:,0],gaps[:,1],c=gaps_col,s=80,zorder=3)

plt.savefig("./Plots/limacon_Chernhierarchy_kpm.png", dpi=150, bbox_inches = 'tight')